In [1]:
import pandas as pd
from IPython.display import display
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import numpy as np

In [2]:
single_report_date = pd.read_csv('mhp_single_report_date.csv')
print(len(single_report_date))
single_report_date.head()

12740


,room_name,refundable_rate,rate_id,stay_date,amount,breakfast,lunch,dinner,adultcount,childcount,infantcount,nights,ota_room_id,rate_update_id,refundable,rate_plan_code,date_update,availability,report_date
0,Classic Double,259.0,51867665159,2024-07-26,259.0,True,False,False,2,0,0,1,606651,34810000022877,True,NaN,2024-07-13 20:00:37,7,2024-07-13
1,Classic Double,199.0,51867665146,2024-07-26,199.0,False,False,False,2,0,0,1,606651,34810000022877,False,NaN,2024-07-13 20:00:37,7,2024-07-13
2,Classic Double,239.0,51867665160,2024-07-26,239.0,True,False,False,1,0,0,1,606651,34810000022877,True,NaN,2024-07-13 20:00:37,7,2024-07-13
3,Classic Double,239.0,51867665155,2024-07-26,239.0,True,False,False,2,0,0,1,606651,34810000022877,False,NaN,2024-07-13 20:00:37,7,2024-07-13
4,Classic Double,219.0,51867665156,2024-07-26,219.0,True,False,False,1,0,0,1,606651,34810000022877,False,NaN,2024-07-13 20:00:37,7,2024-07-13


In [26]:
single_report_date = pd.read_csv('mhp_single_report_date.csv')
unique_stay_dates = single_report_date['stay_date'].nunique()
unique_stay_dates_list = single_report_date['stay_date'].unique().tolist()

print(f"Number of unique stay dates: {unique_stay_dates}")
print(f"List of unique stay dates: {unique_stay_dates_list}")

Number of unique stay dates: 353
List of unique stay dates: ['2024-07-26', '2024-07-28', '2024-07-29', '2024-07-30', '2024-07-31', '2024-08-01', '2024-08-02', '2024-08-03', '2024-08-04', '2024-08-05', '2024-08-09', '2024-08-10', '2024-08-11', '2024-08-12', '2024-08-13', '2024-08-14', '2024-08-15', '2024-08-16', '2024-08-17', '2024-08-18', '2024-08-19', '2024-08-20', '2024-08-21', '2024-08-22', '2024-08-23', '2024-08-24', '2024-08-25', '2024-08-26', '2024-08-27', '2024-08-28', '2024-08-29', '2024-08-30', '2024-08-31', '2024-09-01', '2024-09-02', '2024-09-03', '2024-09-04', '2024-09-05', '2024-09-06', '2024-09-07', '2024-09-08', '2024-09-09', '2024-09-10', '2024-09-11', '2024-09-12', '2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16', '2024-09-17', '2024-09-18', '2024-09-19', '2024-09-20', '2024-09-21', '2024-09-22', '2024-09-23', '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27', '2024-09-28', '2024-09-29', '2024-09-30', '2024-10-01', '2024-10-02', '2024-10-03', '2024-10-04', '

In [44]:
# Define the filter for Classic Double rooms
room_filter = single_report_date['room_name'] == 'Classic Double'

# Apply the filter and sort by refundable rate ascending
classic_double_mhp = single_report_date[room_filter].sort_values(by='refundable_rate', ascending=True)

# Convert columns to integer type
integer_columns = ['breakfast', 'lunch', 'dinner', 'refundable', 'childcount', 'infantcount']
classic_double_mhp[integer_columns] = classic_double_mhp[integer_columns].astype(int)

# Map adultcount
classic_double_mhp['adultcount'] = classic_double_mhp['adultcount'].map({1: 1, 2: 0})

# Select and reorder columns
columns_to_keep = ['room_name', 'stay_date', 'refundable_rate', 'breakfast', 'lunch', 'dinner', 'adultcount', 'childcount', 'infantcount', 'refundable']
classic_double_mhp = classic_double_mhp[columns_to_keep]

# Double-check that only Classic Double rooms are included
classic_double_mhp = classic_double_mhp[classic_double_mhp['room_name'] == 'Classic Double']

# Display the first few rows
display(classic_double_mhp.head())

# Optionally, you can add this line to verify all rows are Classic Double
print(classic_double_mhp['room_name'].unique())

,room_name,stay_date,refundable_rate,breakfast,lunch,dinner,adultcount,childcount,infantcount,refundable
1159,Classic Double,2025-02-09,149.0,0,0,0,0,0,0,0
1118,Classic Double,2025-02-02,149.0,0,0,0,0,0,0,0
1119,Classic Double,2025-02-02,159.0,1,0,0,1,0,0,0
951,Classic Double,2025-01-05,159.0,0,0,0,0,0,0,0
1038,Classic Double,2025-01-20,159.0,0,0,0,0,0,0,0


['Classic Double']


In [53]:
# First, calculate the base rates as before
base_rates = {}
for stay_date in classic_double_mhp['stay_date'].unique():
    date_filter = classic_double_mhp['stay_date'] == stay_date
    base_rate = classic_double_mhp[
        date_filter &
        (classic_double_mhp['breakfast'] == 0) &
        (classic_double_mhp['lunch'] == 0) &
        (classic_double_mhp['dinner'] == 0) &
        (classic_double_mhp['adultcount'] == 0) &
        (classic_double_mhp['childcount'] == 0) &
        (classic_double_mhp['infantcount'] == 0) &
        (classic_double_mhp['refundable'] == 0)
    ]['refundable_rate'].min()
    base_rates[stay_date] = base_rate

# Now, add the base_rate to the DataFrame
classic_double_mhp['base_rate'] = classic_double_mhp['stay_date'].map(base_rates)

# Print the first few rows to verify
print(classic_double_mhp.head())

# Optionally, print the first few items of base_rates to verify
print("\nFirst 5 base rates:")
print(dict(list(base_rates.items())[:5]))

           room_name  stay_date  refundable_rate  breakfast  lunch  dinner  \
1159  Classic Double 2025-02-09            149.0          0      0       0   
1118  Classic Double 2025-02-02            149.0          0      0       0   
1119  Classic Double 2025-02-02            159.0          1      0       0   
951   Classic Double 2025-01-05            159.0          0      0       0   
1038  Classic Double 2025-01-20            159.0          0      0       0   

      adultcount  childcount  infantcount  refundable  base_rate  
1159           0           0            0           0      149.0  
1118           0           0            0           0      149.0  
1119           1           0            0           0      149.0  
951            0           0            0           0      159.0  
1038           0           0            0           0      159.0  

First 5 base rates:
{Timestamp('2025-02-09 00:00:00'): np.float64(149.0), Timestamp('2025-02-02 00:00:00'): np.float64(149.0), T

In [55]:
# Add a column to check if refundable_rate matches base_rate
classic_double_mhp['rate_mismatch'] = classic_double_mhp['refundable_rate'] != classic_double_mhp['base_rate']

# Filter to show only the mismatched rows
mismatched_rates = classic_double_mhp[classic_double_mhp['rate_mismatch']]

# Sort by stay_date for easier reading
mismatched_rates = mismatched_rates.sort_values('stay_date')

# Select relevant columns for display
columns_to_display = ['stay_date', 'refundable_rate', 'base_rate']

# Display the mismatched rates
print(mismatched_rates[columns_to_display])

# Optionally, show the count of mismatched rates
print(f"\nTotal number of mismatched rates: {len(mismatched_rates)}")

      stay_date  refundable_rate  base_rate
4    2024-07-26            219.0      199.0
0    2024-07-26            259.0      199.0
3    2024-07-26            239.0      199.0
2    2024-07-26            239.0      199.0
5    2024-07-26            219.0      199.0
...         ...              ...        ...
2004 2025-06-30            369.0      299.0
2009 2025-06-30            349.0      299.0
2005 2025-06-30            339.0      299.0
2007 2025-06-30            329.0      299.0
2006 2025-06-30            319.0      299.0

[1675 rows x 3 columns]

Total number of mismatched rates: 1675


In [62]:
df = classic_double_mhp

# Ensure boolean variables are coded as 0 and 1
boolean_vars = ['breakfast', 'lunch', 'dinner', 'refundable']
for var in boolean_vars:
    df[var] = df[var].astype(int)

# Create interaction terms
df['breakfast_adult'] = df['breakfast'] * df['adultcount']
df['lunch_adult'] = df['lunch'] * df['adultcount']
df['dinner_adult'] = df['dinner'] * df['adultcount']
df['breakfast_child'] = df['breakfast'] * df['childcount']
df['lunch_child'] = df['lunch'] * df['childcount']
df['dinner_child'] = df['dinner'] * df['childcount']

# Prepare the features
X = df[['breakfast', 'lunch', 'dinner',
        'adultcount', 'childcount', 'infantcount',
        'refundable',
        'breakfast_adult', 'lunch_adult', 'dinner_adult',
        'breakfast_child', 'lunch_child', 'dinner_child']]

# Use base_rate as an offset
y = df['refundable_rate'] - df['base_rate']

# Add a constant term to the features (required for statsmodels OLS regression)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     3435.
Date:                Thu, 12 Sep 2024   Prob (F-statistic):               0.00
Time:                        10:46:32   Log-Likelihood:                -7088.3
No. Observations:                2010   AIC:                         1.418e+04
Df Residuals:                    2006   BIC:                         1.421e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.1866      0.367     

/Users/lohithkonathala/hotelcloud/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [63]:
df = classic_double_mhp

# Ensure boolean variables are coded as 0 and 1
boolean_vars = ['breakfast', 'lunch', 'refundable']
for var in boolean_vars:
    df[var] = df[var].astype(int)

# Prepare the features
X = df[['breakfast', 'lunch',
        'adultcount', 'childcount', 'infantcount',
        'refundable']]

# Use base_rate as an offset
y = df['refundable_rate'] - df['base_rate']

# Add a constant term to the features (required for statsmodels OLS regression)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     3435.
Date:                Thu, 12 Sep 2024   Prob (F-statistic):               0.00
Time:                        10:56:12   Log-Likelihood:                -7088.3
No. Observations:                2010   AIC:                         1.418e+04
Df Residuals:                    2006   BIC:                         1.421e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.1866      0.367     -0.508      

/Users/lohithkonathala/hotelcloud/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
